In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.1.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,546 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://se

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-09-29 21:35:49--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.52MB/s    in 0.2s    

2022-09-29 21:35:49 (5.52 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("house_database").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://mydataclass.s3.us-east-2.amazonaws.com/AmesHousing.csv"
spark.sparkContext.addFile(url)
house_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
house_df.show()

+-----+---------+-----------+---------+------------+--------+------+-----+---------+------------+---------+----------+----------+------------+-----------+-----------+---------+-----------+------------+------------+----------+--------------+----------+---------+------------+------------+------------+------------+----------+----------+----------+---------+---------+-------------+--------------+------------+--------------+------------+-----------+-------------+-------+----------+-----------+----------+----------+----------+---------------+-----------+--------------+--------------+---------+---------+-------------+-------------+------------+-------------+----------+----------+------------+-----------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+------------+-------------+--------------+----------+------------+---------+-------+-----+------------+--------+-------+-------+---------+--------------+---------+
|Order|      PID|MS SubClass|MS Zoning|Lot

In [9]:
# drop Nan values from the df
house_df=house_df.drop("Alley","Pool QC","Fence","Misc Feature")
house_df.show()

+-----+---------+-----------+---------+------------+--------+------+---------+------------+---------+----------+----------+------------+-----------+-----------+---------+-----------+------------+------------+----------+--------------+----------+---------+------------+------------+------------+------------+----------+----------+----------+---------+---------+-------------+--------------+------------+--------------+------------+-----------+-------------+-------+----------+-----------+----------+----------+----------+---------------+-----------+--------------+--------------+---------+---------+-------------+-------------+------------+-------------+----------+----------+------------+-----------+-------------+-------------+-----------+-----------+-----------+-----------+-----------+------------+-------------+--------------+----------+------------+---------+--------+-------+-------+---------+--------------+---------+
|Order|      PID|MS SubClass|MS Zoning|Lot Frontage|Lot Area|Street|Lot Sha

In [12]:
# Configure settings for RDS
mode = "overwrite"
jdbc_url="jdbc:postgresql://housingaws-db.cnvxuytbvp1m.us-east-1.rds.amazonaws.com:5432/housing_database"
config = {"user":"postgres", 
          "password": "****", 
          "driver":"org.postgresql.Driver"}

In [13]:
# Write dataframe to table housing
house_df.write.jdbc(url=jdbc_url, table='housing', mode=mode, properties=config)